In [1]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import rmse_score
from spotlight.factorization.explicit import ExplicitFactorizationModel

dataset = get_movielens_dataset(variant='100K')

train, test = random_train_test_split(dataset)

model = ExplicitFactorizationModel(n_iter=1)
model.fit(train)

rmse = rmse_score(model, test)

Tareas:
1. Mirar y resumir los dsitintos modelos, probarlos
2. Probar distintos tamaños de los modelos
3. Hacer el 

In [6]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel

dataset = get_movielens_dataset(variant='100K')

train, test = random_train_test_split(dataset)

model = ImplicitFactorizationModel(n_iter=3,
                                   loss='bpr')
model.fit(train)

mrr = mrr_score(model, test)
mrr

array([0.03202492, 0.0716944 , 0.01037514, 0.01878123, 0.01610304,
       0.01642852, 0.01244687, 0.02155942, 0.17510959, 0.01055149,
       0.05379548, 0.01619779, 0.00562666, 0.08467118, 0.0195219 ,
       0.01080249, 0.02292705, 0.01191558, 0.00561547, 0.02776584,
       0.01263702, 0.02278307, 0.04842101, 0.01742292, 0.01066422,
       0.02292968, 0.04761905, 0.022419  , 0.05864332, 0.05803743,
       0.00786908, 0.03198042, 0.0254381 , 0.06485849, 0.04781746,
       0.03545492, 0.02331455, 0.01784552, 0.01549267, 0.06360895,
       0.02539714, 0.02004451, 0.01634017, 0.02380777, 0.0715577 ,
       0.03816804, 0.04517496, 0.03717784, 0.01689703, 0.01417521,
       0.02729656, 0.07375249, 0.07151629, 0.01858753, 0.23849561,
       0.00996973, 0.03152683, 0.03078323, 0.01078872, 0.03290581,
       0.04349885, 0.02282841, 0.05921069, 0.0255013 , 0.02431635,
       0.09220936, 0.04421127, 0.28936966, 0.02003372, 0.01266921,
       0.02463053, 0.05991894, 0.04746725, 0.03072244, 0.01786